In [ ]:
import cv2
import pytesseract
import numpy as np
from ultralytics import YOLO
import json
import os
from datetime import datetime

# Конфигурация
CONFIG = {
    "model_path": "yolov8n.pt",
    "image_path": "test.png",  # Убедитесь что файл существует
    "output_dir": "results",
    "ocr_lang": "rus+eng",
    "threshold": 0.5,
    "preprocess": True
}

def preprocess_image(image):
    """Улучшение изображения для OCR"""
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    enhanced = clahe.apply(gray)
    _, thresh = cv2.threshold(enhanced, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    kernel = np.ones((2,2), np.uint8)
    cleaned = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
    return cleaned

def recognize_text(image, lang):
    """Распознавание текста с улучшенными параметрами"""
    if CONFIG["preprocess"]:
        image = preprocess_image(image)
    
    custom_config = r'--oem 3 --psm 6'
    text = pytesseract.image_to_string(
        image, 
        lang=lang,
        config=custom_config
    )
    return text.strip()

def save_results(data, image_with_boxes):
    """Сохранение результатов в файлы"""
    if not os.path.exists(CONFIG["output_dir"]):
        os.makedirs(CONFIG["output_dir"])
    
    # Сохраняем JSON (предварительно конвертируем numpy массивы)
    serializable_data = {
        "image_path": data["image_path"],
        "blocks": [
            {
                "block_id": b["block_id"],
                "bbox": [int(x) for x in b["bbox"]],  # Конвертируем numpy.int64 в int
                "confidence": float(b["confidence"]),  # Конвертируем numpy.float32 в float
                "text": b["text"]
            }
            for b in data["blocks"]
        ]
    }
    
    with open(f"{CONFIG['output_dir']}/results.json", "w", encoding="utf-8") as f:
        json.dump(serializable_data, f, ensure_ascii=False, indent=4)
    
    # Сохраняем изображение с разметкой
    cv2.imwrite(f"{CONFIG['output_dir']/annotated.jpg", image_with_boxes)

def main():
    # Проверка существования файла
    if not os.path.exists(CONFIG["image_path"]):
        raise FileNotFoundError(f"Файл изображения {CONFIG['image_path']} не найден!")
    
    # Загрузка модели
    model = YOLO(CONFIG["model_path"])
    
    # Чтение изображения
    image = cv2.imread(CONFIG["image_path"])
    if image is None:
        raise ValueError(f"Не удалось загрузить изображение {CONFIG['image_path']}")
    
    # Детекция
    results = model.predict(
        source=image,
        conf=CONFIG["threshold"],
        save=False
    )
    
    # Подготовка данных
    output_data = {
        "image_path": CONFIG["image_path"],
        "blocks": []
    }
    annotated = image.copy()
    
    # Обработка результатов
    for i, box in enumerate(results[0].boxes):
        # Координаты
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
        
        # Вырезаем блок
        cropped = image[y1:y2, x1:x2]
        
        # Распознаем текст
        text = recognize_text(cropped, CONFIG["ocr_lang"])
        
        # Визуализация
        cv2.rectangle(annotated, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(
            annotated,
            f"Block {i+1}",
            (x1, y1-10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.7,
            (0,0,255),
            2
        )
        
        # Сохраняем данные
        output_data["blocks"].append({
            "block_id": i+1,
            "bbox": [x1, y1, x2, y2],
            "confidence": float(box.conf[0]),
            "text": text
        })
        
        print(f"\nБлок {i+1}:")
        print(f"Координаты: ({x1}, {y1}) - ({x2}, {y2})")
        print(f"Текст: {text}")
    
    # Сохранение
    save_results(output_data, annotated)
    print(f"\nРезультаты сохранены в {CONFIG['output_dir']}")

if __name__ == "__main__":
    try:
        pytesseract.get_tesseract_version()
    except:
        print("Tesseract не установлен! Скачайте его с https://github.com/tesseract-ocr/tesseract")
        exit(1)
    
    main()

: 

In [1]:
!-m pip install ipykernel -U --force-reinstall

zsh: bad option string: '-m pip install ipykernel -U --force-reinstall'
